In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import gc

from utils.schemas import *
from utils.functions import *

In [2]:
data_folder = 'input'

In [3]:
train = pd.read_csv(data_folder+'/train_generated_0.csv', dtype = schema_generated_0)
test = pd.read_csv(data_folder+'/test_generated_0.csv', dtype = schema_generated_0)

In [4]:
train.shape, test.shape

((590540, 552), (506691, 552))

In [5]:
train_index = train.shape[0]
test_index = test.shape[0]
df = pd.concat([train, test], axis=0)
df.shape

(1097231, 552)

In [6]:
del train
del test
gc.collect()

7

In [7]:
df['Transaction_day_of_week'] = np.floor((df['TransactionDT'] / (3600 * 24) - 1) % 7)
df['Transaction_hour'] = np.floor(df['TransactionDT'] / 3600) % 24

In [8]:
cols1 = [x for x in df.columns if 'card' in x and 'fe' not in x]
group1 = df.groupby(cols1).count()['TransactionDT']
group1.name = 'N1'
df = df.join(group1, on=cols1)

In [9]:
cols1 = [x for x in df.columns if 'addr' in x]
group1 = df.groupby(cols1).count()['TransactionDT']
group1.name = 'N2'
df = df.join(group1, on=cols1)

In [10]:
cols1 = [x for x in df.columns if 'C' in x and 'Product' not in x]
group1 = df.groupby(cols1).count()['TransactionDT'].astype('int64')
group1.name = 'N3'
df = df.join(group1, on=cols1)

In [11]:
cols1 = [x for x in df.columns if 'mail' in x]
group1 = df.groupby(cols1).count()['TransactionDT']
group1.name = 'N4'
df = df.join(group1, on=cols1)

In [12]:
# df = df.drop(['N1', 'N2', 'N3', 'N4'], axis=1)

In [13]:
# for c in ['N1', 'N2', 'N3', 'N4']:
#     df = freq_encoder(df, c, c, 0.005)

In [14]:
from sklearn.cluster import KMeans

In [15]:
n_cls = [2, 4, 8, 16, 32, 64]

In [16]:
cols1 = [x for x in df.columns if 'C' in x and 'Product' not in x]
cols2 = [x for x in df.columns if 'card' in x and 'fe' not in x]
cols3 = [x for x in df.columns if 'V' in x]

cols_group = [cols1, cols2, cols3]

In [18]:
for n in n_cls:
    m = 1
    for cols in cols_group:
        print('Generation feature:\tgroup_{0}_n_{1}'.format(m, n))
        kmeans = KMeans(n_clusters=n, random_state=42).fit(df[cols].fillna(-1))
        df['group_{0}_n_{1}'.format(m, n)] = kmeans.labels_
        m += 1

Generation feature:	group_1_n_2
Generation feature:	group_2_n_2
Generation feature:	group_3_n_2
Generation feature:	group_1_n_4
Generation feature:	group_2_n_4
Generation feature:	group_3_n_4


KeyboardInterrupt: 